In [1]:
# importamos las liberias necesarias para trabajar con python con spark
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, StructField, StructType, IntegerType, FloatType, DateType, BooleanType, DoubleType
from pyspark.sql.functions import *

# creamos la sesion de spark con 4 cores y 4 gigas de memoria mas nombre de la aplicacion
spark = SparkSession.builder.master("local[4]").appName("carga_de_datos").getOrCreate()

# creamos variable con la ruta del archivos
capaProcesamiento = "../datalake/capaProcesamiento/peliculas.parquet"
capaLimpieza = "../datalake/capaLimpieza/peliculas.parquet"

In [3]:
# leamos el df desde la capa de procesamiento
df = spark.read.parquet(capaProcesamiento).persist()

# hacemos un printSchema para ver el esquema del df
df.printSchema()

# hacemos un show para ver los datos del df
df.show(5)

root
 |-- adult: boolean (nullable = true)
 |-- genres: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- imdb_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- release_date: date (nullable = true)
 |-- runtime: double (nullable = true)
 |-- cast: string (nullable = true)
 |-- crew: string (nullable = true)
 |-- keywords: string (nullable = true)
 |-- poster: string (nullable = true)

+-----+--------------------+------+---------+-----------------+--------------------+------------+-------+--------------------+--------------------+--------------------+--------------------+
|adult|              genres|    id|  imdb_id|            title|            overview|release_date|runtime|                cast|                crew|            keywords|              poster|
+-----+--------------------+------+---------+-----------------+--------------------+------------+-------+--------------------+--------------------+---------

In [4]:
# miramos la columna adult
df.select("adult").distinct().show()

+-----+
|adult|
+-----+
| true|
|false|
+-----+



In [13]:
# miramos la columna genres si tiene valores nulos
df.select((col("genres").isNull()) | (col("genres") == "")).groupBy("((genres IS NULL) OR (genres = ))").count().show()

+---------------------------------+-----+
|((genres IS NULL) OR (genres = ))|count|
+---------------------------------+-----+
|                            false|12611|
+---------------------------------+-----+



In [15]:
# miramos la columna id 
df.select("id").distinct().show(5)

# miramos el valor min y max de la columna id
df.select(min("id"), max("id")).show()

+-----+
|   id|
+-----+
|75294|
|87462|
|26755|
|20382|
|30361|
+-----+
only showing top 5 rows

+-------+-------+
|min(id)|max(id)|
+-------+-------+
|      2| 465044|
+-------+-------+



In [17]:
# miramos la columna imdb_id
df.select("imdb_id").distinct().show(5)

# miramos el numero de nulos
df.select((col("imdb_id").isNull()) | (col("imdb_id") == "")).groupBy("((imdb_id IS NULL) OR (imdb_id = ))").count().show()

+---------+
|  imdb_id|
+---------+
|tt4126394|
|tt0098153|
|tt0039305|
|tt0093780|
|tt0964185|
+---------+
only showing top 5 rows

+-----------------------------------+-----+
|((imdb_id IS NULL) OR (imdb_id = ))|count|
+-----------------------------------+-----+
|                              false|12611|
+-----------------------------------+-----+



In [18]:
# miramos la columna title
df.select("title").distinct().show(5)

+------------------+
|             title|
+------------------+
|The Last Airbender|
|   My Name Is Khan|
|            Crisis|
|     The Dead Girl|
|     The Encounter|
+------------------+
only showing top 5 rows



In [22]:
# miramos la columna overview
df.select("overview").distinct().show(5)

# comprobamos si no tiene valores int
df.filter(df["overview"].rlike("^[0-9]+$")).show()

+--------------------+
|            overview|
+--------------------+
|After a group of ...|
|A freak tsunami t...|
|Born of a god but...|
|With friends like...|
|Hamlet comes home...|
+--------------------+
only showing top 5 rows

+-----+------+---+-------+-----+--------+------------+-------+----+----+--------+------+
|adult|genres| id|imdb_id|title|overview|release_date|runtime|cast|crew|keywords|poster|
+-----+------+---+-------+-----+--------+------------+-------+----+----+--------+------+
+-----+------+---+-------+-----+--------+------------+-------+----+----+--------+------+



In [24]:
# miramos la columna keywords
df.select("keywords").distinct().show(5)

# miramos la cantidad de valores con []
df.filter(df["keywords"].rlike("^\[\]$")).count()

+--------------------+
|            keywords|
+--------------------+
|[{'id': 10051, 'n...|
|[{'id': 1329, 'na...|
|[{'id': 567, 'nam...|
|[{'id': 10540, 'n...|
|[{'id': 6270, 'na...|
+--------------------+
only showing top 5 rows



3072

In [38]:
# comprobamos si la columna poster tiene url validas y que empiieza por https://
print(df.filter(df["poster"].rlike("^https://")).count())

# comprobamos si la columna poster tiene url validas y que no empiieza por https://
print(df.filter(~df["poster"].rlike("^https://")).select("poster").show(5, truncate=False))

df = df.filter(df["poster"].rlike("^https://"))

df.select("poster").show(5, truncate=False)


8796
+-----------------+
|poster           |
+-----------------+
| 'id': 27737     |
| {'cast_id': 1018|
| 'id': 14984     |
| 'id': 1288498   |
| 'id': 6798      |
+-----------------+
only showing top 5 rows

None
+------------------------------------------------------------------------------------------------------------------------------------------------+
|poster                                                                                                                                          |
+------------------------------------------------------------------------------------------------------------------------------------------------+
|https://m.media-amazon.com/images/M/MV5BMTY0Y2EyYTMtMDM5MC00YjEzLWFhY2QtMmRhZmFlNWI3Nzg1XkEyXkFqcGdeQXVyMjUyNDk2ODc@._V1_UX256,0,256,256_AL_.jpg|
|https://m.media-amazon.com/images/M/MV5BN2MyYTZiZTktZTBlNC00MDJhLThmOGEtZTc3NmE2ZmQ2Y2U4XkEyXkFqcGdeQXVyMTE4ODk5NjI@._V1_UX256,0,256,256_AL_.jpg|
|https://m.media-amazon.com/images/M/MV5BODgxNzI5N

In [39]:
# miramos el numero de nulos por columna
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-----+------+---+-------+-----+--------+------------+-------+----+----+--------+------+
|adult|genres| id|imdb_id|title|overview|release_date|runtime|cast|crew|keywords|poster|
+-----+------+---+-------+-----+--------+------------+-------+----+----+--------+------+
|    0|     0|  0|      0|    0|       0|           0|      0|   0|   0|       0|     0|
+-----+------+---+-------+-----+--------+------------+-------+----+----+--------+------+



In [40]:
# miramos el numero de registros 
df.count()

8796

In [41]:
# camiamos el tipo de datos de la columna release_date a string
df = df.withColumn("release_date", col("release_date").cast(StringType()))

# miramos el numero de nulos por columna
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()


+-----+------+---+-------+-----+--------+------------+-------+----+----+--------+------+
|adult|genres| id|imdb_id|title|overview|release_date|runtime|cast|crew|keywords|poster|
+-----+------+---+-------+-----+--------+------------+-------+----+----+--------+------+
|    0|     0|  0|      0|    0|       0|           0|      0|   0|   0|       0|     0|
+-----+------+---+-------+-----+--------+------------+-------+----+----+--------+------+



In [42]:
# miramos la columna release_date
df.select("release_date").distinct().show(5)

+------------+
|release_date|
+------------+
|  2001-04-22|
|  2008-12-03|
|  2010-02-12|
|  2013-03-14|
|  2007-04-29|
+------------+
only showing top 5 rows



In [43]:
# guadamos el df en la capa de limpieza
df.write.mode("overwrite").parquet(capaLimpieza)

# liberamos memoria
df.unpersist()

In [44]:
# cerramos la sesion de spark
spark.stop()